# Viewing an ABI band

Without downloading the data from the cloud. All processing is done on Amazon Web Services.

This code is inspired by Amy Huff and by Mya Sears: 
* https://github.com/modern-tools-workshop/AMS-python-workshop-2023/blob/main/download_satellite_data.ipynb
* https://gitlab.cicsnc.org/workshop-development/ams-short-course/-/blob/main/course_materials/02_GOES_aws_classroom.ipynb?ref_type=head


In [1]:
import s3fs
import requests
import datetime
from pathlib import Path
import xarray as xr
import fnmatch
import netCDF4
import numpy as np
import requests

In [2]:
directory_path = Path.cwd()

Hooking up the AWS S3 buckets:

In [3]:
fs = s3fs.S3FileSystem(anon=True)

Viewing the products available:

In [4]:
bucket = 'noaa-goes16'

products_path = bucket

products = fs.ls(products_path)

for product in products:
    print(product.split('/')[-1])

ABI-L1b-RadC
ABI-L1b-RadF
ABI-L1b-RadM
ABI-L2-ACHA2KMC
ABI-L2-ACHA2KMF
ABI-L2-ACHA2KMM
ABI-L2-ACHAC
ABI-L2-ACHAF
ABI-L2-ACHAM
ABI-L2-ACHP2KMC
ABI-L2-ACHP2KMF
ABI-L2-ACHP2KMM
ABI-L2-ACHTF
ABI-L2-ACHTM
ABI-L2-ACMC
ABI-L2-ACMF
ABI-L2-ACMM
ABI-L2-ACTPC
ABI-L2-ACTPF
ABI-L2-ACTPM
ABI-L2-ADPC
ABI-L2-ADPF
ABI-L2-ADPM
ABI-L2-AICEF
ABI-L2-AITAF
ABI-L2-AODC
ABI-L2-AODF
ABI-L2-BRFC
ABI-L2-BRFF
ABI-L2-BRFM
ABI-L2-CCLC
ABI-L2-CCLF
ABI-L2-CCLM
ABI-L2-CMIPC
ABI-L2-CMIPF
ABI-L2-CMIPM
ABI-L2-COD2KMF
ABI-L2-CODC
ABI-L2-CODF
ABI-L2-CPSC
ABI-L2-CPSF
ABI-L2-CPSM
ABI-L2-CTPC
ABI-L2-CTPF
ABI-L2-DMWC
ABI-L2-DMWF
ABI-L2-DMWM
ABI-L2-DMWVC
ABI-L2-DMWVF
ABI-L2-DMWVM
ABI-L2-DSIC
ABI-L2-DSIF
ABI-L2-DSIM
ABI-L2-DSRC
ABI-L2-DSRF
ABI-L2-DSRM
ABI-L2-FDCC
ABI-L2-FDCF
ABI-L2-FDCM
ABI-L2-FSCC
ABI-L2-FSCF
ABI-L2-FSCM
ABI-L2-LSAC
ABI-L2-LSAF
ABI-L2-LSAM
ABI-L2-LST2KMF
ABI-L2-LSTC
ABI-L2-LSTF
ABI-L2-LSTM
ABI-L2-LVMPC
ABI-L2-LVMPF
ABI-L2-LVMPM
ABI-L2-LVTPC
ABI-L2-LVTPF
ABI-L2-LVTPM
ABI-L2-MCMIPC
ABI-L2-MCMIPF
ABI-L2-MCMIPM
ABI

In [5]:
year = 2023
month = 9
day = 7

julian_day = datetime.datetime(year, month, day).strftime('%j')
print(julian_day)

250


Get the files from the day selected:

In [6]:
bucket = 'noaa-goes16'
product = 'ABI-L1b-RadF'
year = 2023
julian = 250
hour = 0

data_path = bucket + '/' + product + '/'  + str(year) + '/' + str(julian).zfill(3) + '/' + str(hour).zfill(2)

files = fs.ls(data_path)

print('Total number of files:', len(files), '\n')

for file in files[:10]:
    print(file.split('/')[-1])
print('...')
for file in files[-10:]:
    print(file.split('/')[-1])

Total number of files: 96 

OR_ABI-L1b-RadF-M6C01_G16_s20232500000203_e20232500009511_c20232500009554.nc
OR_ABI-L1b-RadF-M6C01_G16_s20232500010203_e20232500019511_c20232500019556.nc
OR_ABI-L1b-RadF-M6C01_G16_s20232500020203_e20232500029511_c20232500029565.nc
OR_ABI-L1b-RadF-M6C01_G16_s20232500030203_e20232500039511_c20232500039538.nc
OR_ABI-L1b-RadF-M6C01_G16_s20232500040203_e20232500049511_c20232500049560.nc
OR_ABI-L1b-RadF-M6C01_G16_s20232500050203_e20232500059512_c20232500059558.nc
OR_ABI-L1b-RadF-M6C02_G16_s20232500000203_e20232500009511_c20232500009547.nc
OR_ABI-L1b-RadF-M6C02_G16_s20232500010203_e20232500019511_c20232500019545.nc
OR_ABI-L1b-RadF-M6C02_G16_s20232500020203_e20232500029511_c20232500029546.nc
OR_ABI-L1b-RadF-M6C02_G16_s20232500030203_e20232500039511_c20232500039544.nc
...
OR_ABI-L1b-RadF-M6C15_G16_s20232500020203_e20232500029518_c20232500029552.nc
OR_ABI-L1b-RadF-M6C15_G16_s20232500030203_e20232500039517_c20232500039564.nc
OR_ABI-L1b-RadF-M6C15_G16_s20232500040203_e2

In [7]:
files_07 = [file for file in files if fnmatch.fnmatchcase(file.split('/')[-1], "OR_ABI-L1b-RadF-M6C07*")]

In [8]:
aws = s3fs.S3FileSystem(anon=True)

In [9]:
fnames = []

satellite = 'noaa-goes16'
product = 'ABI-L1b-RadF'
scan = ''
year = '2023'
doy = '250'
hour = '00'

data_files = aws.ls(f'{satellite}/{product}/{year}/{doy}/{hour}/', refresh=True) 

In [10]:
for file in data_files:
    file_start = file.split('_')[-3][8:12]
    file_end = file.split('_')[-2][8:12]
    fnames.append(file)

In [11]:
fnames[0]

'noaa-goes16/ABI-L1b-RadF/2023/250/00/OR_ABI-L1b-RadF-M6C01_G16_s20232500000203_e20232500009511_c20232500009554.nc'

In [12]:
resp = requests.get(f'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadF/2023/250/00/OR_ABI-L1b-RadF-M6C01_G16_s20232500000203_e20232500009511_c20232500009554.nc')

In [13]:
resp

<Response [200]>

In [14]:
nc = netCDF4.Dataset(fnames[0], memory = resp.content)

In [15]:
ds = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))

In [16]:
ds

<xarray.Dataset>
Dimensions:                                           (y: 10848, x: 10848,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       band: 1,
                                                       number_of_harmonization_coefficients: 3,
                                                       num_star_looks: 24)
Coordinates:
    t                                                 datetime64[ns] ...
  * y                                                 (y) float64 0.1519 ... ...
  * x                                                 (x) float64 -0.1519 ......
    y_image                                           float32 ...
    x_image                                           float32 ...
    band_id                                           (band) int8 ...
    band_wavelength                                   (band) float32 ...
    t_star_look                                       (num_star_looks) datetime64[ns] ...
    band_wavelength_star_look                         (num_star_looks) float32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band, number_of_harmonization_coefficients,
                                num_star_looks
Data variables: (12/39)
    Rad                                               (y, x) float32 ...
    DQF                                               (y, x) float32 ...
    time_bounds                                       (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                            int32 ...
    y_image_bounds                                    (number_of_image_bounds) float32 ...
    x_image_bounds                                    (number_of_image_bounds) float32 ...
    ...                                                ...
    algorithm_dynamic_input_data_container            int32 ...
    processing_parm_version_container                 int32 ...
    algorithm_product_version_container               int32 ...
    star_id                                           (num_star_looks) float32 ...
    channel_integration_time                          float64 ...
    channel_gain_field                                float64 ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           WCDAS
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2023-09-07T00:09:55.4Z
    time_coverage_start:       2023-09-07T00:00:20.3Z
    time_coverage_end:         2023-09-07T00:09:51.1Z
    LUT_Filenames:             SpaceLookParams(FM1A_ADR1270_PR_12_02_01)-7387...
    id:                        c94f67b9-9588-42b5-ba5c-5ce43a5f375e

Might be able to run create_band07_func.py to create the lat/lon dimensions: 